In [1]:
import sys, time, random, math, cv2, os
import numpy as np

sys.path.insert(0, '../src')
from robot import Robot

In [2]:
path_gtg = "Q_matrix_gtg.txt"
path_ao  = "Q_matrix_ao.txt"
on_disk = path_gtg in os.listdir() and path_ao in os.listdir()

actions = np.array([(3,3), (1,-1), (-1,1)])
N_STATE_GTG = 120

dimensions_gtg = (N_STATE_GTG, len(actions))
dimensions_ao = (4, len(actions))

if on_disk:
    f_gtg = open(path_gtg, "r")    
    t_gtg = f_gtg.read().split()
    _ = float(t_gtg[0])
    random_rate_gtg = float(t_gtg[1])
    _ = float(t_gtg[2])
    Q_gtg = np.reshape(np.array([float(i) for i in t_gtg[3:]]), dimensions_gtg)
    f_gtg.close()

    f_ao = open(path_ao, "r")    
    t_ao = f_ao.read().split()
    _ = float(t_gtg[0])
    random_rate_ao = float(t_ao[1])
    _ = float(t_ao[2])
    Q_ao = np.reshape(np.array([float(i) for i in t_ao[3:]]), dimensions_ao)
    f_ao.close()

else:
    print("Files not found ;(")
    exit(1)


In [3]:
def cicle(ang):
    return (ang + 2 * math.pi) % (2*math.pi)

def get_distance_from_goal(a,b):
    return np.linalg.norm(a-b)

def get_angular_distance(v1,v2):

    v1_aux = v1 / np.linalg.norm(v1)
    v2_aux = v2 / np.linalg.norm(v2)

    theta = np.arccos(np.clip(np.dot(v1_aux, v2_aux), -1.0, 1.0))
    rot_theta = np.array([[np.cos(theta),-np.sin(theta)],[np.sin(theta),np.cos(theta)]])
    theta = theta*180.0/math.pi
    theta = (360.0+theta)%360.0

    if abs(np.dot(np.dot(rot_theta,v2_aux),v1_aux)-1.0) > 1e-2:
        theta = 360.0 - theta

    return theta

def get_state_go_to_goal(current_position, current_angle, goal):
    angle_distance = get_angular_distance(goal - current_position, np.array([math.cos(current_angle),math.sin(current_angle)]))
    if angle_distance <= (180 / N_STATE_GTG) or angle_distance >= (360 - (180 / N_STATE_GTG)):
        r = 0
    else:
        r = (angle_distance - (180 / N_STATE_GTG)) / (360 / N_STATE_GTG) + 1
    return int(r)

def get_action_go_to_goal(Q, state, random_rate):
    a = random.random()
    if a < random_rate:
        i = random.randint(0, len(actions) - 1)
    else:
        i = 0
        for j in range(len(actions)):
            if Q[state][j] > Q[state][i]:
                i = j
    return i

In [4]:
def get_state_avoid_obstacle(us_distances):
    threshold = 0.5
    r = 0
    r += 1 * (us_distances[3] <= threshold)
    r += 2 * (us_distances[4] <= threshold)
    
    return r

def get_action_avoid_obstacle(Q, state, random_rate):
    a = random.random()
    if a < random_rate:
        i = random.randint(0, len(actions) - 1)
    else:
        i = 0
        for j in range(len(actions)):
            if Q[state][j] > Q[state][i]:
                i = j
    return i

In [5]:
robot = Robot("#0")

goal = np.array([(13,0), (-13,0)])
i = 0

score = 0

while robot.get_connection_status() != -1:
    us_distances = robot.read_ultrassonic_sensors()

    current_position = np.array(robot.get_current_position()[:-1])
    current_angle = cicle(robot.get_current_orientation()[2])
    
    state_gtg = get_state_go_to_goal(current_position, current_angle, goal[i])
    state_ao = get_state_avoid_obstacle(us_distances)
    
    flag = False
    for j in us_distances[:8]:
        flag = flag or (j < 0.5)

    if not flag:
        vel = actions[get_action_go_to_goal(Q_gtg, state_gtg, random_rate_gtg)]
    else:
        vel = actions[get_action_avoid_obstacle(Q_ao, state_ao, random_rate_ao)]

    robot.set_left_velocity(vel[0] * 2)
    robot.set_right_velocity(vel[1] * 2)
    time.sleep(0.1)
    
    if get_distance_from_goal(current_position, goal[i]) < 0.3:
        i = (i + 1) % 2
        score += 1


Connected to remoteApi server.


KeyboardInterrupt: 